# lorem.ai - Synthetic Database Data Generator

> Lorem.ai is a powerful database data generator that creates realistic synthetic data for testing and development purposes. It analyzes your database schema and sample data to intelligently generate high-quality synthetic records that maintain the statistical properties, relationships, and patterns of your original data. Using advanced AI models, Lorem.ai can identify and preserve data distributions, text patterns, and case formatting while ensuring data privacy. Perfect for creating test datasets, development environments, and data anonymization without compromising data utility or realism.

This notebook demonstrates how to use Lorem.ai to generate synthetic data for a database table.

![image](logo.jpg)

## 1. Setup and Imports

First, let's import the necessary modules and functions from the main.py file.

In [17]:
%%bash
pip install -q ollama numpy pandas

In [38]:
import json
import os
import sqlite3
import pandas as pd

# Import the required functions from main.py
from src.db import DatabaseExtractor

# Import the functions we need from main.py
from main import (
    build_graph,
    get_db_path,
    validate_graph,
    generate_synthetic_data,
    print_json
)

## 2. Configuration

Let's set up the configuration for our data generation task.

In [33]:
# Configuration
table_name = "users"  # The table we want to generate data for
num_rows = 100000     # Number of rows to generate
max_retries = 5       # Maximum validation retries
output_db_path = "synthetic_users.sqlite"  # Output database path

# Display configuration
print(f"Configuration:")
print(f"  - Table: {table_name}")
print(f"  - Rows to generate: {num_rows:,}")
print(f"  - Output database: {output_db_path}")
print(f"  - Maximum validation retries: {max_retries}")

Configuration:
  - Table: users
  - Rows to generate: 100,000
  - Output database: synthetic_users.sqlite
  - Maximum validation retries: 5


## 3. Extract Table Information

Now, let's extract information about the table structure and sample data from the original database.

In [34]:
# Get SQLite database path
db_path = get_db_path()
if db_path is None:
    print("❌ Error: Could not find the database path. Please run generate_test_db.py first.")
else:
    print(f"✅ Found database at: {db_path}")

# Create DatabaseExtractor instance
extractor = DatabaseExtractor(db_path=db_path)

# Get table information
print(f"\n=== Extracting table information for {table_name} ===")
try:
    table_info = extractor.get_table_info(table_name)
    print(f"  - Found {len(table_info['columns'])} columns")
    print(f"  - Extracted {len(table_info['sample_data'])} sample rows")
    
    # Display column information
    print("\nColumn information:")
    for i, column in enumerate(table_info['columns']):
        col_name = column.get('column_name', '')
        data_type = column.get('data_type', '')
        nullable = "NULL" if column.get('is_nullable', 'YES') == 'YES' else "NOT NULL"
        print(f"  {i+1}. {col_name} ({data_type}, {nullable})")
    
    # Get detailed statistics for numeric columns
    print("\nAnalyzing numeric columns...")
    numeric_stats = {}
    for column in table_info['columns']:
        col_name = column.get('column_name', '')
        data_type = column.get('data_type', '').lower()
        if data_type in ['integer', 'real', 'numeric', 'decimal', 'float', 'double']:
            print(f"  - Getting statistics for {col_name}...")
            try:
                stats = extractor.get_numeric_column_stats(table_name, col_name)
                numeric_stats[col_name] = stats
                print(f"    Min: {stats.get('min')}, Max: {stats.get('max')}, Mean: {stats.get('mean'):.2f}")
            except Exception as e:
                print(f"    Error analyzing {col_name}: {e}")
    
    print("\nSample data (pretty format):")
    df = pd.DataFrame(table_info['sample_data'])
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', 1000)
    display(df)
except Exception as e:
    print(f"❌ Error extracting table information: {e}")

✅ Found database at: db/test_db.sqlite

=== Extracting table information for users ===
  - Found 15 columns
  - Extracted 5 sample rows

Column information:
  1. id (INTEGER, NULL)
  2. first_name (TEXT, NOT NULL)
  3. last_name (TEXT, NOT NULL)
  4. email (TEXT, NOT NULL)
  5. phone_number (TEXT, NULL)
  6. address (TEXT, NULL)
  7. city (TEXT, NULL)
  8. state (TEXT, NULL)
  9. zipcode (TEXT, NULL)
  10. country (TEXT, NULL)
  11. job_title (TEXT, NULL)
  12. company (TEXT, NULL)
  13. username (TEXT, NULL)
  14. date_of_birth (DATE, NULL)
  15. created_at (TIMESTAMP, NULL)

Analyzing numeric columns...
  - Getting statistics for id...
    Min: 1, Max: 10, Mean: 5.50

Sample data (pretty format):


,id,first_name,last_name,email,phone_number,address,city,state,zipcode,country,job_title,company,username,date_of_birth,created_at
0,1,Danielle,Johnson,john21@example.net,001 581 896 0013,9402 Peterson Drives,Port Matthew,Colorado,50298,Bhutan,Mechanical engineer,Munoz-Roman,yherrera,1975-05-13,2025-02-26 03:15:10
1,2,Andrea,Calderon,barbara10@example.net,441 731 6475,41928 Frank Light Suite 835,East Lydiamouth,Montana,35594,Bahrain,"Engineer, mining","Walter, Edwards and Rios",pwilkerson,2001-03-23,2025-02-26 03:15:10
2,3,Douglas,Taylor,julie69@example.com,(332)887-1012,784 Erin Crescent Suite 514,Mooretown,Washington,94830,Estonia,"Restaurant manager, fast food",Romero Inc,meagan89,1983-07-15,2025-02-26 03:15:10
3,4,Eric,Farmer,georgetracy@example.org,(443)903-9117,8963 Jennifer Locks,Samuelhaven,New Hampshire,16361,Egypt,Equality and diversity officer,Morgan PLC,briannasmith,2018-03-03,2025-02-26 03:15:10
4,5,Crystal,Perez,heathchad@example.org,373-382-9973,6670 Bruce Stream Suite 133,New Shane,Nevada,24551,Fiji,"Psychologist, counselling",Baxter Inc,jmorton,1976-08-30,2025-02-26 03:15:10


## 4. Generate Initial Graph

Now, let's generate the initial graph using the LLM. The graph defines how to generate synthetic data for each column.

In [35]:
# Generate graph using LLM
print(f"\n=== Generating initial graph for {table_name} ===")
try:
    graph_result = build_graph(
        table_name=table_info['table_name'],
        columns=table_info['columns'],
        sample_data=table_info['sample_data']
    )
    
    # Parse the result
    graph_data = json.loads(graph_result)
    print("\n=== Generated Graph ===")
    print_json(graph_data)
    
    # Display a summary of the graph
    print("\n=== Graph Summary ===")
    for column in graph_data.get('columns', []):
        col_name = column.get('name', '')
        graph_keys = column.get('graph', [])
        mapping = column.get('mapping', 'none')
        col_type = column.get('type', 'string')
        print(f"Column: {col_name}")
        print(f"  - Graph keys: {', '.join(graph_keys)}")
        print(f"  - Mapping: {mapping}")
        print(f"  - Type: {col_type}")
        print()
except json.JSONDecodeError:
    print("❌ Error: Could not parse LLM response as JSON")
    print("Raw response:")
    print(graph_result)
except Exception as e:
    print(f"❌ Error generating graph: {e}")


=== Generating initial graph for users ===
  - Preparing to build graph for table 'users'
  - Loaded 64 faker key descriptions
  - Created prompt with 15 columns and 5 sample rows
  - Sending request to LLM...
  - Using Ollama for graph generation
  - Received response from LLM (1303 characters)

=== Generated Graph ===
{
  "columns": [
    {
      "name": "id",
      "graph": [
        "integer"
      ],
      "mapping": "none",
      "type": "numeric"
    },
    {
      "name": "first_name",
      "graph": [
        "first_name"
      ],
      "mapping": "titlecase",
      "type": "string"
    },
    {
      "name": "last_name",
      "graph": [
        "last_name"
      ],
      "mapping": "titlecase",
      "type": "string"
    },
    {
      "name": "email",
      "graph": [
        "email"
      ],
      "mapping": "lowercase",
      "type": "string"
    },
    {
      "name": "phone_number",
      "graph": [
        "phone_number"
      ],
      "mapping": "none",
      "type":

## 5. Validate and Improve the Graph

Now, let's validate the graph by generating a small sample of data and comparing it to the original data. If there are issues, we'll try to improve the graph.

In [22]:
# Track retry count
retry_count = 0
is_valid = False

# Store all versions of the graph for later selection
graph_versions = {
    "original": graph_data
}

while not is_valid and retry_count < max_retries:
    print(f"\n=== Validation attempt {retry_count + 1}/{max_retries} ===")
    
    # Validate the current graph with a small sample
    print(f"  - Generating 5 sample rows for validation...")
    is_valid, validation_report, improved_graph, feedback = validate_graph(
        graph_data=graph_data,
        table_name=table_name,
        original_db_path=db_path,
        sample_size=5  # Generate 5 rows for validation
    )
    
    if is_valid:
        print("✅ Validation successful! The graph produces data that matches the original patterns.")
        break
    
    print("❌ Validation found issues with the generated data.")
    
    # If we have an improved graph from automatic fixes
    if improved_graph:
        print("\n=== Applying automatic fixes to the graph ===")
        print("  - The system has automatically identified and fixed some issues:")
        
        # Print what was fixed
        for col_name, col_info in validation_report["columns"].items():
            if col_info["issues"]:
                for issue in col_info["issues"]:
                    print(f"    * {col_name}: {issue}")
        
        # Store this version
        graph_versions["auto_fixed"] = improved_graph
        
        # Update the current graph
        graph_data = improved_graph
        
        # Validate the improved graph
        print("\n=== Validating improved graph ===")
        print(f"  - Generating 5 sample rows for validation...")
        is_valid, validation_report, _, _ = validate_graph(
            graph_data=improved_graph,
            table_name=table_name,
            original_db_path=db_path,
            sample_size=5
        )
        
        if is_valid:
            print("✅ Validation successful after automatic fixes!")
            break
        else:
            print("❌ Automatic fixes were not sufficient to resolve all issues.")
    
    # If we still have issues and have feedback for the LLM
    if not is_valid and feedback:
        print("\n=== Rebuilding graph with LLM using validation feedback ===")
        print("  - Sending the following feedback to the LLM:")
        for line in feedback.split('\n'):
            print(f"    {line}")
        
        # Rebuild the graph with feedback
        print("  - Waiting for LLM to generate improved graph...")
        new_graph_result = build_graph(
            table_name=table_info['table_name'],
            columns=table_info['columns'],
            sample_data=table_info['sample_data'],
            feedback=feedback
        )
        
        try:
            new_graph_data = json.loads(new_graph_result)
            print("\n=== LLM Rebuilt Graph ===")
            print_json(new_graph_data)
            
            # Store this version
            graph_versions["llm_rebuilt"] = new_graph_data
            
            # Update the current graph
            graph_data = new_graph_data
            
            # Validate the new graph
            print("\n=== Validating LLM rebuilt graph ===")
            print(f"  - Generating 5 sample rows for validation...")
            is_valid, validation_report, _, _ = validate_graph(
                graph_data=new_graph_data,
                table_name=table_name,
                original_db_path=db_path,
                sample_size=5
            )
            
            if is_valid:
                print("✅ Validation successful after LLM rebuilding!")
                break
            else:
                print("❌ LLM rebuilding did not resolve all issues.")
                print("  - Remaining issues:")
                for issue in validation_report["issues"]:
                    print(f"    * {issue}")
        
        except json.JSONDecodeError:
            print("❌ Error: Could not parse LLM rebuilt graph as JSON")
            print("  - Raw response:")
            print(new_graph_result)
    
    # Increment retry count
    retry_count += 1
    
    if retry_count < max_retries and not is_valid:
        print(f"\n=== Retry {retry_count + 1}/{max_retries} ===")
        print("  - Previous validation attempt failed.")
        print("  - Attempting another validation cycle...")


=== Validation attempt 1/5 ===
  - Generating 5 sample rows for validation...

=== Validating graph with 5 sample rows ===
  - Generating synthetic sample data...
Extracting statistics for numeric column: id
  - Stats: min=1, max=10, mean=5.5, stddev=2.8722813232690143
  - Using only valid keys: 
  - No valid keys remain, using default key: number

Continuing with valid keys only...

Creating table users...
Generating 5 rows of synthetic data...
  Inserted rows 1 to 5
Successfully generated 5 rows of synthetic data in temp_validation.sqlite
  - Successfully generated sample data
  - Comparing with original data...
  - Retrieved 5 original rows and 5 synthetic rows
  - Analyzing data patterns by column:
  - Error validating graph: could not convert string to float: b'\x05\x00\x00\x00\x00\x00\x00\x00'
  - Cleaned up temporary validation database
❌ Validation found issues with the generated data.

=== Retry 2/5 ===
  - Previous validation attempt failed.
  - Attempting another validation

## 6. Generate Synthetic Data

Now, let's generate the full synthetic dataset using the selected graph.

In [36]:
# Generate synthetic data
print(f"\n=== Generating {num_rows:,} rows of synthetic data for {table_name} ===")
print(f"  - Using the {'validated' if is_valid else 'selected'} graph")
print(f"  - Output database: {output_db_path}")
print(f"  - This may take a while for large datasets...")

# Create output directory if it doesn't exist
output_dir = os.path.dirname(output_db_path)
if output_dir and not os.path.exists(output_dir):
    os.makedirs(output_dir)
    print(f"  - Created output directory: {output_dir}")

success = generate_synthetic_data(
    graph_data=graph_data,
    table_name=table_name,
    num_rows=num_rows,
    output_db_path=output_db_path,
    original_db_path=db_path  # Pass the original database path for numeric stats
)

if success:
    print(f"\n=== Data generation completed successfully ===")
    print(f"  - Generated {num_rows:,} rows for table '{table_name}'")
    print(f"  - Output database: {output_db_path}")
else:
    print("\n=== Data generation failed ===")
    print("  - Check the error messages above for details.")


=== Generating 100,000 rows of synthetic data for users ===
  - Using the selected graph
  - Output database: synthetic_users.sqlite
  - This may take a while for large datasets...
Extracting statistics for numeric column: id
  - Stats: min=1, max=10, mean=5.5, stddev=2.8722813232690143
  - Using only valid keys: 
  - No valid keys remain, using default key: number

Continuing with valid keys only...

Creating table users...
Generating 100000 rows of synthetic data...
  Inserted rows 1 to 1000
  Inserted rows 1001 to 2000
  Inserted rows 2001 to 3000
  Inserted rows 3001 to 4000
  Inserted rows 4001 to 5000
  Inserted rows 5001 to 6000
  Inserted rows 6001 to 7000
  Inserted rows 7001 to 8000
  Inserted rows 8001 to 9000
  Inserted rows 9001 to 10000
  Inserted rows 10001 to 11000
  Inserted rows 11001 to 12000
  Inserted rows 12001 to 13000
  Inserted rows 13001 to 14000
  Inserted rows 14001 to 15000
  Inserted rows 15001 to 16000
  Inserted rows 16001 to 17000
  Inserted rows 17001

## 7. Viewing Results

Let's examine the synthetic data we've generated and compare it with the original data to ensure quality.

In [37]:
# Load and display a sample of the synthetic data
print("\n=== Viewing Generated Synthetic Data ===")

# Connect to the output database
conn_synth = sqlite3.connect(output_db_path)

# Get the total row count in the synthetic database
row_count = pd.read_sql_query(f"SELECT COUNT(*) FROM {table_name}", conn_synth).iloc[0, 0]
print(f"Total rows in synthetic database: {row_count:,}")

# Query the synthetic data (excluding id column)
synth_df = pd.read_sql_query(f"SELECT * FROM {table_name} LIMIT 10", conn_synth)
synth_df = synth_df.iloc[:, 1:]  # Skip the id column

# Display the synthetic data
print(f"\nSynthetic data sample ({table_name}):")
display(synth_df)

# Compare with original data
print("\n=== Comparing with Original Data ===")

# Connect to the original database
conn_orig = sqlite3.connect(db_path)

# Query the original data (excluding id column)
orig_df = pd.read_sql_query(f"SELECT * FROM {table_name} LIMIT 10", conn_orig)
orig_df = orig_df.iloc[:, 1:]  # Skip the id column

# Display the original data
print(f"\nOriginal data sample ({table_name}):")
display(orig_df)

# Close connections
conn_synth.close()
conn_orig.close()



=== Viewing Generated Synthetic Data ===
Total rows in synthetic database: 100,000

Synthetic data sample (users):


,first_name,last_name,email,phone_number,address,city,state,zipcode,country,job_title,company,username,date_of_birth,created_at
0,Nichole,Hopkins,angelawashington@example.net,214-376-1234x499,913 Beck Creek Suite 498,Johnberg,KY,00951,Sierra Leone,Corporate Treasurer,"Brooks, Spencer And Hill",qphillips,52.09,1509368789
1,Melissa,Jordan,escott@example.net,(821)294-5586x9577,148 Flores Mountain Suite 802,Allenton,FM,53028,Syrian Arab Republic,"Education Officer, Environmental",Rogers-Howard,joshuarodriguez,53.65,354434159
2,John,Clark,alexanderdaniel@example.net,+1-950-516-9436x814,3509 Keith Flats Suite 850,West Natashamouth,IN,89964,Armenia,Chief Operating Officer,Washington Llc,hayesstacy,59.38,20761629
3,Jeffrey,Ortiz,mwiggins@example.org,(878)900-7141,03315 Jacob Ford Apt. 612,Lawrenceville,PW,25379,Micronesia,Building Control Surveyor,"Stein, Heath And Rodriguez",laurieschwartz,42.47,878851563
4,Joshua,Yu,timothysanchez@example.com,001-683-572-1620x7386,227 Sean Extension Suite 382,Morganport,OR,85956,Korea,Holiday Representative,Buck Llc,petersenchelsea,63.02,906216967
5,Bailey,James,richardwilliams@example.org,599-405-2008,45501 Holden Harbor,Lake Tiffanyview,TN,42015,Antigua And Barbuda,Charity Fundraiser,"Watts, Jones And Anderson",david46,45.27,944611127
6,Brett,Hughes,bowenrichard@example.com,861.725.5683x89797,05260 Christina Parkways Apt. 548,Tylerport,PA,73620,New Caledonia,Careers Adviser,Booker Group,kristi58,47.45,1454519516
7,Mitchell,Spencer,nwebb@example.org,+1-263-697-2550,30102 Jarvis View Apt. 482,North Mario,ID,95837,Burundi,"Clinical Scientist, Histocompatibility And Imm...",Kirk-Thomas,amy09,37.34,847074801
8,Lindsay,Banks,ricemegan@example.com,283-625-5085,2567 Munoz Spring Suite 516,New Kylefort,CA,16036,Christmas Island,Public Relations Officer,Wilkerson-Bell,stevenboyer,54.79,1582792089
9,Ashley,Wilson,pamela20@example.org,698.255.0544x50409,4830 Luna Orchard Apt. 852,East Alexisshire,AS,86098,Ecuador,Merchant Navy Officer,Hall Ltd,gthompson,39.42,1415287318



=== Comparing with Original Data ===

Original data sample (users):


,first_name,last_name,email,phone_number,address,city,state,zipcode,country,job_title,company,username,date_of_birth,created_at
0,Danielle,Johnson,john21@example.net,001 581 896 0013,9402 Peterson Drives,Port Matthew,Colorado,50298,Bhutan,Mechanical engineer,Munoz-Roman,yherrera,1975-05-13,2025-02-26 03:15:10
1,Andrea,Calderon,barbara10@example.net,441 731 6475,41928 Frank Light Suite 835,East Lydiamouth,Montana,35594,Bahrain,"Engineer, mining","Walter, Edwards and Rios",pwilkerson,2001-03-23,2025-02-26 03:15:10
2,Douglas,Taylor,julie69@example.com,(332)887-1012,784 Erin Crescent Suite 514,Mooretown,Washington,94830,Estonia,"Restaurant manager, fast food",Romero Inc,meagan89,1983-07-15,2025-02-26 03:15:10
3,Eric,Farmer,georgetracy@example.org,(443)903-9117,8963 Jennifer Locks,Samuelhaven,New Hampshire,16361,Egypt,Equality and diversity officer,Morgan PLC,briannasmith,2018-03-03,2025-02-26 03:15:10
4,Crystal,Perez,heathchad@example.org,373-382-9973,6670 Bruce Stream Suite 133,New Shane,Nevada,24551,Fiji,"Psychologist, counselling",Baxter Inc,jmorton,1976-08-30,2025-02-26 03:15:10
5,Brent,Kidd,novaksara@example.org,(267)873-6026,4687 James Forge Apt. 980,Brianshire,New Mexico,66410,Uruguay,Set designer,Brown PLC,yorkcasey,1974-08-17,2025-02-26 03:15:10
6,Emma,Young,davidashley@example.net,+1-509-616-9985,47510 Howell Port Apt. 183,Davidstad,California,32519,Heard Island and McDonald Islands,Higher education careers adviser,Kim PLC,cheryl80,145929600000,2025-02-26 03:15:10
7,Lisa,Evans,mcmillandennis@example.org,682-944-9353,16400 Wise Route Suite 278,Hoffmanville,California,91074,United Kingdom,Database administrator,"Johnson, Wood and Tran",tanderson,1994-10-25,2025-02-26 03:15:10
8,Rachel,Flynn,nolanjason@example.org,+1-815-586-9232,16073 Lisa Mount Suite 433,Lake Jamesmouth,Louisiana,09600,Sierra Leone,Health promotion specialist,May-Ross,sheila14,1982-02-25,2025-02-26 03:15:10
9,Beverly,Terrell,rodriguezryan@example.net,569-781-6934,59514 Estrada Islands Suite 482,South Elizabeth,Missouri,89278,San Marino,Ecologist,"Carroll, Sullivan and Bass",justin69,2011-04-14,2025-02-26 03:15:10
